In [0]:
SET use_cached_result = false;
SELECT  *
FROM    (
            SELECT  i_category
                    ,i_class
                    ,i_brand
                    ,s_store_name
                    ,s_company_name
                    ,d_moy
                    ,SUM(ss_sales_price) sum_sales
                    ,AVG(SUM(ss_sales_price)) OVER (PARTITION BY i_category,i_brand,s_store_name,s_company_name ) avg_monthly_sales
            FROM workspace.tpcds_sf1000_iceberg.item
            ,       workspace.tpcds_sf1000_iceberg.store_sales
            ,       workspace.tpcds_sf1000_iceberg.date_dim
            ,       workspace.tpcds_sf1000_iceberg.store
            WHERE   ss_item_sk = i_item_sk
            AND     ss_sold_date_sk = d_date_sk
            AND     ss_store_sk = s_store_sk
            AND     d_year IN (1999)
            AND     (
                        (
                                    i_category IN ('Jewelry','Men','Children')
                                    AND     i_class IN ('loose stones','accessories','school-uniforms')
                        )
                        OR      (
                                    i_category IN ('Home','Shoes','Books')
                                    AND     i_class IN ('bathroom','kids','history')
                        )
            )
            GROUP BY i_category
                      ,i_class
                      ,i_brand
                      ,s_store_name
                      ,s_company_name
                      ,d_moy
        ) tmp1
WHERE   CASE   WHEN (
                    avg_monthly_sales <> 0
        ) THEN (
                    abs(sum_sales - avg_monthly_sales) / avg_monthly_sales
        )
        ELSE NULL
END > 0.1
ORDER BY sum_sales - avg_monthly_sales,s_store_name
LIMIT   100
;